In [ ]:
import numpy as np 
import pandas as pd
import os
import joblib
import cv2

## Preprocessing

In [ ]:
path = r'D:\dbda\project\FER Model\UTKFace'
files = os.listdir(path)
size = len(files)

In [ ]:
images = []
ages = []
genders = []
for i,file in enumerate(files):
  ################
    image = cv2.imread(path+'/'+file,0)
    image = cv2.resize(image,dsize=(64,64))
    image = image.reshape((image.shape[0],image.shape[1],1))
    images.append(image)
    split_var = file.split('_')
    ages.append(split_var[0])
    genders.append(int(split_var[1]))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
ages_array = np.asarray(ages,dtype=np.int)

In [ ]:
scaler = MinMaxScaler()
ages_scaled = scaler.fit_transform(ages_array.reshape((-1,1)))

In [ ]:
target = np.zeros((size,2),dtype='float32')
features = np.zeros((size,images[0].shape[0],images[0].shape[1],1),dtype = 'float32')
for i in range(size):
    target[i,0] = ages_scaled[i][0]
    target[i,1] = int(genders[i])
    features[i] = images[i]
features = features / 255

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2,shuffle  = True)
print("Samples in Training:",x_train.shape[0])
print("Samples in Testing:",x_test.shape[0])

## Training Model

In [ ]:
import keras 
from keras.layers import *
from keras.models import *
from keras import backend as K

In [ ]:
inputs = Input(shape=(64,64,1))
conv1 = Conv2D(32, kernel_size=(3, 3),activation='relu')(inputs)
conv2 = Conv2D(64, kernel_size=(3, 3),activation='relu')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(128, kernel_size=(3, 3),activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv3)
x = Dropout(0.25)(pool2)
flat = Flatten()(x)

dropout = Dropout(0.5)
age_model = Dense(128, activation='relu')(flat)
age_model = dropout(age_model)
age_model = Dense(64, activation='relu')(age_model)
age_model = dropout(age_model)
age_model = Dense(32, activation='relu')(age_model)
age_model = dropout(age_model)
age_model = Dense(1, activation='relu')(age_model)

dropout = Dropout(0.5)
gender_model = Dense(128, activation='relu')(flat)
gender_model = dropout(gender_model)
gender_model = Dense(64, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(32, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(16, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(8, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(1, activation='sigmoid')(gender_model)

In [ ]:
model = Model(inputs=inputs, outputs=[age_model,gender_model])
model.compile(optimizer = 'adam', loss =['mse','binary_crossentropy'],metrics=['accuracy'])

In [ ]:
h = model.fit(x_train,[y_train[:,0],y_train[:,1]],validation_data=(x_test,[y_test[:,0],y_test[:,1]]),epochs = 10, batch_size=128,shuffle = True)

## Saving Trained Model

In [ ]:
joblib.dump(scaler,"scaler.pkl")

In [ ]:
model_json = model.to_json()
with open("age&gender_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("age&gender_model.h5")